In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re

In [2]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)

In [3]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel("F:/Empresas SF/reportes/aaaaaa.xlsx", index=False)

In [4]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape


(7757, 8)

In [5]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


(7447, 5)

In [6]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(7763, 10)

In [7]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

fin empresas y comienzo de rescate de pacientes


In [8]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape


(1820412, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [9]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape


(1435047, 4)

In [10]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1435047, 13)

In [11]:
empresas_unidas.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [67]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [68]:
# pacientes_empresas22.shape

In [69]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [70]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [71]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [77]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Pesco"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [78]:
Pacientes_Archivo.head(10)


,Empresa,Rut Titular,rut1g,rut1,sexo,Parentesco,nombres,apellidos,nacimiento,email,direccion,comuna,celular,ciudad
0,PEÑA SPOERER,16.926.710-3,16.926.710-3,16926710-3,M,NaN,MADELAINE,MOLINA,1988-09-16 00:00:00,mcms.molina@gmail.com,Topater #2481,ANTOFAGASTA,9 5003 8158,ANTOFAGASTA
1,PEÑA SPOERER,26.975.847-3,26.975.847-3,26975847-3,M,NaN,CARLOS,CASADIEGO,1991-06-16 00:00:00,carloscasadiego14@gmail.com,Belisario Prats #1850,SANTIAGO,9 3777 4682,SANTIAGO
2,PEÑA SPOERER,17.073.823-3,17.073.823-3,17073823-3,M,NaN,SAUL,CARRION,1989-04-29 00:00:00,saulcarrionrsk@gmail.com,Avenida Recoleta #4112,SANTIAGO,9 8715 6152,SANTIAGO
3,PEÑA SPOERER,20.066.773-5,20.066.773-5,20066773-5,M,NaN,CAMILA,GUERRERO,16-07-1999,camilaguerrero54@gmail.com,"Trapen KM 1031, Camino a Pargua",PTO MONTT,9 7188 6013,PTO MONTT
4,PEÑA SPOERER,18.825.456-k,18.825.456-k,18825456-k,M,NaN,FRANCISCO,CRUZ,1994-04-04 00:00:00,fco.cruz0494@gmail.com,Florida #2977,CALAMA,9 9548 4832,CALAMA
5,PEÑA SPOERER,18.825.456-k,18.825.456-k,18825456-k,M,NaN,HARAM,VALLADARES,1993-02-08 00:00:00,haram.valladares@gmail.com,Oficina Lastenia #11975 - Block G - Depto. 22,ANTOFAGASTA,9 9472 4622,ANTOFAGASTA
6,PEÑA SPOERER,18.344.404-2,18.344.404-2,18344404-2,M,NaN,ERICK,SAN MARTÍN,1992-10-21 00:00:00,ericksanmartin81@gmail.com,Parcela #16 - Virquenco,LOS ANGELES,9 3704 4824,LOS ANGELES
7,PESCO RENTAL,19.562.859-9,19.562.859-9,19562859-9,M,NaN,CRISTOPHER,LUCERO,1996-12-19 00:00:00,cristopherlucerop@gmail.com,Avenida Las Nieves Oriente #1133,SANTIAGO,9 5939 0213,SANTIAGO
8,PEÑA SPOERER,19.385.141-k,19.385.141-k,19385141-k,M,NaN,RAIMUNDO,ALEGRÍA,1996-08-12 00:00:00,raimundohansalegria@gmail.com,Rio Serrano # 03710,LA PINTANA,9 8254 4648,LA PINTANA
9,PEÑA SPOERER,19.385.141-k,19.385.141-k,19385141-k,M,NaN,BENJAMIN,FUENZALIDA,1988-06-06 00:00:00,bfuenzag@gmail.com,Pasaje 1 Poniente - Parcela 27 D,SANTIAGO,9 9846 7226,SANTIAGO


In [79]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [80]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()


,Empresa,Rut Titular,rut1g,rut1,sexo,Parentesco,nombres,apellidos,nacimiento,email,direccion,comuna,celular,ciudad,rut
0,PEÑA SPOERER,16.926.710-3,16.926.710-3,16926710-3,M,NaN,MADELAINE,MOLINA,1988-09-16 00:00:00,mcms.molina@gmail.com,Topater #2481,ANTOFAGASTA,9 5003 8158,ANTOFAGASTA,169267103
1,PEÑA SPOERER,26.975.847-3,26.975.847-3,26975847-3,M,NaN,CARLOS,CASADIEGO,1991-06-16 00:00:00,carloscasadiego14@gmail.com,Belisario Prats #1850,SANTIAGO,9 3777 4682,SANTIAGO,269758473
2,PEÑA SPOERER,17.073.823-3,17.073.823-3,17073823-3,M,NaN,SAUL,CARRION,1989-04-29 00:00:00,saulcarrionrsk@gmail.com,Avenida Recoleta #4112,SANTIAGO,9 8715 6152,SANTIAGO,170738233
3,PEÑA SPOERER,20.066.773-5,20.066.773-5,20066773-5,M,NaN,CAMILA,GUERRERO,16-07-1999,camilaguerrero54@gmail.com,"Trapen KM 1031, Camino a Pargua",PTO MONTT,9 7188 6013,PTO MONTT,200667735
4,PEÑA SPOERER,18.825.456-k,18.825.456-k,18825456-k,M,NaN,FRANCISCO,CRUZ,1994-04-04 00:00:00,fco.cruz0494@gmail.com,Florida #2977,CALAMA,9 9548 4832,CALAMA,18825456k


In [81]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10876\3578335771.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Empresa,Rut Titular,rut1g,rut1,sexo,Parentesco,nombres,apellidos,nacimiento,email,direccion,comuna,celular,ciudad,rut
0,PEÑA SPOERER,16.926.710-3,16.926.710-3,16926710-3,M,NaN,MADELAINE,MOLINA,1988-09-16 00:00:00,mcms.molina@gmail.com,Topater #2481,ANTOFAGASTA,9 5003 8158,ANTOFAGASTA,169267103
1,PEÑA SPOERER,26.975.847-3,26.975.847-3,26975847-3,M,NaN,CARLOS,CASADIEGO,1991-06-16 00:00:00,carloscasadiego14@gmail.com,Belisario Prats #1850,SANTIAGO,9 3777 4682,SANTIAGO,269758473
2,PEÑA SPOERER,17.073.823-3,17.073.823-3,17073823-3,M,NaN,SAUL,CARRION,1989-04-29 00:00:00,saulcarrionrsk@gmail.com,Avenida Recoleta #4112,SANTIAGO,9 8715 6152,SANTIAGO,170738233
3,PEÑA SPOERER,20.066.773-5,20.066.773-5,20066773-5,M,NaN,CAMILA,GUERRERO,16-07-1999,camilaguerrero54@gmail.com,"Trapen KM 1031, Camino a Pargua",PTO MONTT,9 7188 6013,PTO MONTT,200667735
4,PEÑA SPOERER,18.825.456-k,18.825.456-k,18825456-k,M,NaN,FRANCISCO,CRUZ,1994-04-04 00:00:00,fco.cruz0494@gmail.com,Florida #2977,CALAMA,9 9548 4832,CALAMA,18825456K


In [82]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10876\2830316430.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10876\2830316430.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [83]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(20, 25)

In [84]:
Pacientes_con_id.head()


,Empresa,Rut Titular,rut1g,rut1,sexo,Parentesco,nombres,apellidos,nacimiento,email,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,PEÑA SPOERER,16.926.710-3,16.926.710-3,16926710-3,M,NaN,Madelaine,Molina,1988-09-16 00:00:00,mcms.molina@gmail.com,...,True,0018c00002nkxBCAAY,169267103,Cecilia Madelaine Molina Selti,RUT,None,None,None,Mujer,950038158
1,PEÑA SPOERER,26.975.847-3,26.975.847-3,26975847-3,M,NaN,Carlos,Casadiego,1991-06-16 00:00:00,carloscasadiego14@gmail.com,...,True,001RN000000QSSAYA4,269758473,Carlos Casadiego,RUT,None,None,None,None,937774682
2,PEÑA SPOERER,17.073.823-3,17.073.823-3,17073823-3,M,NaN,Saul,Carrion,1989-04-29 00:00:00,saulcarrionrsk@gmail.com,...,True,0018c00002eUPtBAAW,170738233,Saul Patricio Carrion Castillo,RUT,None,None,None,Hombre,None
3,PEÑA SPOERER,20.066.773-5,20.066.773-5,20066773-5,M,NaN,Camila,Guerrero,16-07-1999,camilaguerrero54@gmail.com,...,True,001RN000000QSSBYA4,200667735,Camila Guerrero,RUT,None,None,None,None,97188613
4,PEÑA SPOERER,18.825.456-k,18.825.456-k,18825456-k,M,NaN,Francisco,Cruz,1994-04-04 00:00:00,fco.cruz0494@gmail.com,...,True,0018c00002bHgnbAAC,18825456K,Francisco Gabri Cruz Cruz,RUT,None,None,None,Hombre,None


In [85]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [86]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (Pacientes_con_id["email"].astype(str).str.lower())
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(lambda y: isValid(y))
    #Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    #Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10876\3808489073.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
7,False,Cristopher,Lucero,195628599,cristopherlucerop@gmail.com,1996-12-19 00:00:00,Hombre,avenida las nieves oriente #1133,Santiago,Santiago,9 5939 0213,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
8,False,Raimundo,Alegría,19385141K,raimundohansalegria@gmail.com,1996-08-12 00:00:00,Hombre,rio serrano # 03710,La Pintana,La Pintana,9 8254 4648,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
9,False,Benjamin,Fuenzalida,19385141K,bfuenzag@gmail.com,1988-06-06 00:00:00,Hombre,pasaje 1 poniente - parcela 27 d,Santiago,Santiago,9 9846 7226,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
10,False,Eduardo,Arias,188800920,eduardo.arias.marin@gmail.com,1994-07-12 00:00:00,Hombre,el agua quieta # 1801,Santiago,Santiago,9 2093 4496,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
11,False,Cristian,Zamora,19165420K,cristianmzamorap@gmail.com,1995-12-11 00:00:00,Hombre,calle canela # 0514 - villa la reina,Santiago,Santiago,9 3877 6978,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [87]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [88]:
""" print(dia_hoy) """


' print(dia_hoy) '

In [89]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
158600,a1Y8c00000C6VeNEAV,0018c00002amRRLAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Peña Spoerer Y Cía S.A.,Sin Holding Asociado,Pesco,Peña Spoerer Y Cía S.A.,Sin Póliza Asociada,None,EMP7399,A-0723
158601,a1Y8c00000C6VeOEAV,0018c00002amRzAAAU,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Peña Spoerer Y Cía S.A.,Sin Holding Asociado,Pesco,Peña Spoerer Y Cía S.A.,Sin Póliza Asociada,None,EMP7399,A-0723
158602,a1Y8c00000C6VePEAV,0018c00002amLUtAAM,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Peña Spoerer Y Cía S.A.,Sin Holding Asociado,Pesco,Peña Spoerer Y Cía S.A.,Sin Póliza Asociada,None,EMP7399,A-0723
158603,a1Y8c00000C6VeQEAV,0018c00002amSFBAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Peña Spoerer Y Cía S.A.,Sin Holding Asociado,Pesco,Peña Spoerer Y Cía S.A.,Sin Póliza Asociada,None,EMP7399,A-0723
158604,a1Y8c00000C6VeREAV,0018c00002amSFFAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Peña Spoerer Y Cía S.A.,Sin Holding Asociado,Pesco,Peña Spoerer Y Cía S.A.,Sin Póliza Asociada,None,EMP7399,A-0723


In [90]:
campaña_seleccionada.shape


(420, 13)

In [91]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )


In [92]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )


In [93]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [94]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)
